To Do 9/25/24
1. Convert Term Data to 2,30, and 40 codes be all 30!!!
2. retention rate (changing major in earlier versus later semesters)
3. display in df: College - retention rate (no regard to major change) - (rentention rate (regard to those who change major))

Questions:
1. are the colleges entered for all students? The array looks like the cell below.
array(['ED', nan, 'LA', 'SM', 'FA', 'FR', 'BU'], dtype=object)

2. where are graduating students being accounted for?

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

outcome_df = pd.read_csv('data/new_outcome.csv')

outcome_df['college_bt'].unique()

# convert terms that end in 25 or 40 to end with 30
def convert_terms(major_history):
    terms = major_history.split('|')
    converted_terms = []
    for term in terms:
        if '25' in term or '40' in term:
            converted_terms.append(term.replace('25', '30').replace('40', '30'))
        else:
            converted_terms.append(term)
    return '|'.join(converted_terms)

# Update the major_history column
outcome_df['major_history'] = outcome_df['major_history'].apply(convert_terms)

In [2]:
outcome_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73878 entries, 0 to 73877
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   encrypt_id                    73878 non-null  int64  
 1   term                          73878 non-null  int64  
 2   transfer_hours_bt             73878 non-null  int64  
 3   total_sfa_hours_bt            73878 non-null  int64  
 4   total_sfa_hours_et            73878 non-null  int64  
 5   begin_of_semester_gpa         60480 non-null  float64
 6   semester_gpa                  61835 non-null  float64
 7   end_of_term_gpa               70646 non-null  float64
 8   college_bt                    71106 non-null  object 
 9   department_bt                 71106 non-null  object 
 10  major_bt                      71106 non-null  object 
 11  minor_bt                      19044 non-null  object 
 12  concentration_bt              34809 non-null  object 
 13  c

October 5th - Analysis

In [3]:

# Detect changes in college
outcome_df['college_changed'] = outcome_df.apply(lambda row: 1 if row['college_bt'] != row['college_et'] else 0, axis=1)

def parse_major_history(major_history):
    history_items = major_history.split(',')
    return [item.strip() for item in history_items]

outcome_df['parsed_major_history'] = outcome_df['major_history'].apply(parse_major_history)

college_changes = outcome_df[outcome_df['college_bt'] != outcome_df['college_et']].copy()

# Create a new column to check if major has changed
college_changes.loc[:, 'major_changed'] = college_changes.apply(lambda row: 1 if row['major_bt'] != row['major_et'] else 0, axis=1)

# Select relevant columns for display
result = college_changes[['encrypt_id', 'term', 'college_bt', 'college_et', 'major_bt', 'major_et', 'major_changed']]

# Display the result
print(result)

       encrypt_id    term college_bt college_et major_bt major_et  \
8        30130199  202220         LA         ED     COMS     KINE   
13       30127807  202310         LA         SM     CONC     NURS   
18       30126638  202230         FA         SM     MUSC     CISY   
36       30112192  202510        NaN         FR      NaN     AGEN   
56       30101509  202510        NaN         ED      NaN     INDS   
...           ...     ...        ...        ...      ...      ...   
73854       10070  202310         BU         ED     SPBM     INDS   
73856        6068  202510        NaN         LA      NaN     CONC   
73857        4405  202510        NaN         FR      NaN     AGEN   
73860        3952  202410         LA         ED     CONC     KINE   
73875        2154  202210         LA         SM     CONC     NURS   

       major_changed  
8                  1  
13                 1  
18                 1  
36                 1  
56                 1  
...              ...  
73854     

October 21st
- added a column for classification in outcome_df
- classifying students based on the credit hours that they have transferred and through sfa

In [4]:
# make new column with apply lam - gives classification by credit hours (or binning)
outcome_df['total_hours'] = outcome_df['transfer_hours_bt'] + outcome_df['total_sfa_hours_bt']
#print(outcome_df['total_hours'])

#function that classfies students based on the number of credit hours
def classify_student(total_hours):
    if total_hours < 30:
        return 'Freshman'
    elif 30 <= total_hours < 60:
        return 'Sophomore'
    elif 60 <= total_hours < 90:
        return 'Junior'
    else:
        return 'Senior'

#for every student id apply the classification function
outcome_df['Classification'] = outcome_df['total_hours'].apply(classify_student)
print(outcome_df['Classification'])

0           Senior
1           Senior
2           Senior
3           Senior
4           Senior
           ...    
73873    Sophomore
73874     Freshman
73875     Freshman
73876     Freshman
73877     Freshman
Name: Classification, Length: 73878, dtype: object


October 18th - Discontinuation Rates for ALL vs College Changing Students

In [5]:
import numpy as np

# Ensure 'term' is a string
outcome_df['term'] = outcome_df['term'].astype(str)

# Filter out rows where 'term' is either '202420' or '202510'
outcome_df = outcome_df[(outcome_df['term'] != '202510')]

# Calculate discontinued rate for all students using function given
disc = outcome_df.groupby(['college_et', 'term']).agg({'encrypt_id': 'count', 'discontinued': 'sum'})
disc['all_disc_rate'] = (disc['discontinued'] / disc['encrypt_id'] * 100).round(2)  # Round to 2 decimal places

#print dataframe with disc rate for all students
print("The discontinued rate for any college regardless of college switches.")
print(disc)

# Set the change column based on college_bt and college_et (considering only those who switched colleges)
outcome_df['change'] = np.where(
    outcome_df['college_bt'].isna() | outcome_df['college_et'].isna(), 0,  # No change if either is NaN
    np.where(outcome_df['college_bt'] != outcome_df['college_et'], 1, 0)  # Compare normally otherwise
)

# Print rows where 'change' is equal to 1 (students who switched colleges)
changed_rows = outcome_df[outcome_df['change'] == 1]
print(changed_rows[['encrypt_id', 'college_bt', 'college_et', 'change']])

# Discontinued rate for only those who switched colleges
print("The discontinued rate of only those who switched colleges.")
changed_students_df = outcome_df[outcome_df['change'] == 1]

# Calculate discontinued rate for students who switched colleges
changed_students_df = changed_students_df.groupby(['college_et', 'term']).agg({'encrypt_id': 'count', 'discontinued': 'sum'})
changed_students_df['change_disc_rate'] = (changed_students_df['discontinued'] / changed_students_df['encrypt_id'] * 100).round(2)
print(changed_students_df)

print("Both the data for ALL Students within a college and ONLY those who switched colleges.")
# Merge the two DataFrames on 'college_et' and 'term'
merged_disc = pd.merge(disc, changed_students_df, on=['college_et', 'term'], how='left')
merged_disc = merged_disc[['all_disc_rate', 'change_disc_rate']]
print(merged_disc)

The discontinued rate for any college regardless of college switches.
                   encrypt_id  discontinued  all_disc_rate
college_et term                                           
BU         202210        1609           166          10.32
           202220        1401           161          11.49
           202230         620            57           9.19
           202310        1472           141           9.58
           202320        1288           142          11.02
           202330         570            47           8.25
           202410        1404           126           8.97
           202420        1260           153          12.14
           202430         561            43           7.66
ED         202210        2610           196           7.51
           202220        2338           211           9.02
           202230        1034            45           4.35
           202310        2445           186           7.61
           202320        2179           197  

In [6]:
import numpy as np
import pandas as pd

# Create the total hours column
outcome_df['total_hours'] = outcome_df['transfer_hours_bt'] + outcome_df['total_sfa_hours_bt']

# Function that classifies students based on the number of credit hours
def classify_student(total_hours):
    if total_hours < 30:
        return 'Freshman'
    elif 30 <= total_hours < 60:
        return 'Sophomore'
    elif 60 <= total_hours < 90:
        return 'Junior'
    else:
        return 'Senior'

# Apply the classification function
outcome_df['Classification'] = outcome_df['total_hours'].apply(classify_student)

# Ensure 'term' is a string
outcome_df['term'] = outcome_df['term'].astype(str)

# Filter out rows where 'term' is either '202420' or '202510'
outcome_df = outcome_df[(outcome_df['term'] != '202510')]

# Calculate discontinued rate for all students
disc = outcome_df.groupby(['college_et', 'term']).agg({'encrypt_id': 'count', 'discontinued': 'sum'})
disc['all_disc_rate'] = (disc['discontinued'] / disc['encrypt_id'] * 100).round(2)

# Set the change column to indicate college switch status
outcome_df['change'] = np.where(
    outcome_df['college_bt'].isna() | outcome_df['college_et'].isna(), 0,  # No change if either is NaN
    np.where(outcome_df['college_bt'] != outcome_df['college_et'], 1, 0)  # Compare otherwise
)

# Filter rows for students who switched colleges
changed_rows = outcome_df[outcome_df['change'] == 1]

# Calculate discontinued rate for students who switched colleges
changed_students_df = changed_rows.groupby(['college_et', 'term']).agg({'encrypt_id': 'count', 'discontinued': 'sum'})
changed_students_df['change_disc_rate'] = (changed_students_df['discontinued'] / changed_students_df['encrypt_id'] * 100).round(2)

# Merge the discontinued rate data for all students and only those who switched colleges
merged_disc = pd.merge(disc, changed_students_df, on=['college_et', 'term'], how='left')

# Select only the relevant discontinued rate columns for display
merged_disc = merged_disc[['all_disc_rate', 'change_disc_rate']]
print("Discontinued rates for all students and only those who switched colleges:")
print(merged_disc)


Discontinued rates for all students and only those who switched colleges:
                   all_disc_rate  change_disc_rate
college_et term                                   
BU         202210          10.32              6.35
           202220          11.49             16.67
           202230           9.19             20.83
           202310           9.58             18.56
           202320          11.02             14.29
           202330           8.25              5.88
           202410           8.97             18.00
           202420          12.14             32.56
           202430           7.66             13.64
ED         202210           7.51             10.39
           202220           9.02             15.94
           202230           4.35              3.77
           202310           7.61              8.44
           202320           9.04             13.18
           202330           5.90              9.09
           202410           6.57              5.37
        

In [17]:
# Ensure 'term' is a string
outcome_df['term'] = outcome_df['term'].astype(str)

# Filter out rows where 'term' is '202510'
outcome_df = outcome_df[(outcome_df['term'] != '202510')]

# Calculate discontinued rate for all students by classification
disc_by_classification = outcome_df.groupby(['Classification', 'term']).agg({'encrypt_id': 'count', 'discontinued': 'sum'})
disc_by_classification['disc_rate'] = (disc_by_classification['discontinued'] / disc_by_classification['encrypt_id'] * 100).round(2)
print(disc_by_classification['encrypt_id'])

# Create a pivot table for discontinued rate by classification and term
disc_pivot = disc_by_classification.pivot_table(values='disc_rate', index='term', columns='Classification')
print("Discontinued rates by classification and term:")
print(disc_pivot)

Classification  term  
Freshman        202210    2809
                202220    1915
                202230     386
                202310    2985
                202320    2121
                202330     391
                202410    3064
                202420    2117
                202430     374
Junior          202210    2497
                202220    2346
                202230     978
                202310    2249
                202320    2135
                202330     881
                202410    2136
                202420    2017
                202430     782
Senior          202210    3020
                202220    2952
                202230    1464
                202310    2846
                202320    2787
                202330    1303
                202410    2619
                202420    2574
                202430    1249
Sophomore       202210    1890
                202220    1758
                202230     531
                202310    1662
                

In [8]:
# Freshman is significantly higher in discontinued rate than the other categories
# Sophomore seems to flucuate depending on the semester
# Junior and senior lower more significantly
#trend with freshman being most likely to discontinue (older you get, more likely to stay)
#pivot table seperating different gpa groups (4.0-3.5, 3.5-3.0, 3.0-2.0, 2.0 or less)
#is it true with all colleges? 
#combine all falls, springs, and summers? look at 3 categories and each college 

In [ ]:
#find the majors in order that each student has switched from/to
outcome_df['major_history'] = outcome_df['major_history'].fillna('') # Ensure that 'major_history' is not NaN
outcome_df['major_list'] = outcome_df['major_history'].apply(lambda x: [entry.split()[0] for entry in x.split('|')]) #split major history by '|' to get list of past majors
students_with_multiple_majors = outcome_df[outcome_df['major_list'].apply(lambda x: len(x) > 1)] # Filter for students with more than one major
majors_in_order = students_with_multiple_majors.groupby('encrypt_id')['major_list'].apply(lambda x: ', '.join(x.iloc[0])).reset_index() # Group by 'encrypt_id' to list majors in order for each student
majors_in_order.columns = ['encrypt_id', 'majors_in_order']
print(majors_in_order) #get and display columns

In [ ]:
import pandas as pd

def display_major_changes_with_terms(outcome_df):
  
    #get input
    student_id = input("Enter student ID: ")

    # Validate the input
    if not student_id.isdigit():
        print("Invalid student ID. Try again.")
        return

    id = int(student_id)  # Convert to integer

    # Filter the DataFrame for the specific student ID
    student_data = outcome_df[outcome_df['encrypt_id'] == id]

    if student_data.empty:
        print(f"No data found for student ID: {id}")
        return

    # Get the major history
    major_history = student_data['major_history'].iloc[0] #get major history
    major_list = [entry.split()[0] for entry in major_history.split('|')] #get major info
    term_list = [int(entry.split('(')[1].strip(')')) for entry in major_history.split('|')] #get term info
    
    # Create a DataFrame for results
    results = {
        'Term': [],
        'Semester': [],
        'Major': [],
        'Term Difference': []
    }

    # Calculate term differences
    for i in range(len(major_list)):
        results['Major'].append(major_list[i]) #display each major
        results['Term'].append(term_list[i]) #display the term of major
    
        if i == 0:
            results['Term Difference'].append(0)  # No difference for the first major (term 1)
            results['Semester'].append(1)
        else:
            # Convert term to term number
            current_term_number = 3 * (term_list[i] // 100) + (term_list[i] % 100) // 10
            #print(current_term_number)
            first_term_number = 3 * (term_list[0] // 100) + (term_list[0] % 100) // 10
            #print(first_term_number)
        
            # Calculate the term difference
            term_diff = current_term_number - first_term_number
            #print(term_diff)
            results['Term Difference'].append(term_diff)
            results['Semester'].append(term_diff + 1)
            results_df = pd.DataFrame(results)
    return results_df

# Example usage
display_major_changes_with_terms(outcome_df)

In [ ]:
display_major_changes_with_terms(outcome_df)

In [ ]:
def calculate_retention_and_withdrawal_rate(df, term_start, term_end):
    print("First Method: For all students of SFA.")
    # Find students enrolled at the start of the term
    students_start = df[(df['term'] == term_start)]
    total_enrolled_start = len(students_start)

    # Find students enrolled at the end of the term
    students_end = df[df['term'] == term_end]

    # Calculate retained students
    retained_students = students_end[students_end['encrypt_id'].isin(students_start['encrypt_id'])]
    total_retained = len(retained_students)

    # Calculate withdrew students
    withdrew_students = students_end[students_end['withdrew'] == 1]  # Assuming '1' indicates a withdrawal
    total_withdrew = len(withdrew_students)
    discontinued_students = students_end[students_end['discontinued'] == 1]
    total_discontinued = len(discontinued_students)

    # Calculate retention rate
    retention_rate = (total_retained / (total_enrolled_start - total_discontinued)) * 100 if total_enrolled_start > 0 else 0 

    return retention_rate, total_enrolled_start, total_retained, total_withdrew, total_discontinued

# Example usage
start_term = 202110  # Change this to the semester you want to analyze
end_term = 202120    # Change this to the semester you want to analyze
retention_rate, total_enrolled_start, total_retained, total_withdrew, total_discontinued = calculate_retention_and_withdrawal_rate(outcome_df, start_term, end_term)

# Display the results
print(f"Retention Rate: {retention_rate:.2f}%")
print(f"Total Enrolled at Start: {total_enrolled_start}")
print(f"Total Retained: {total_retained}")
print(f"Total Withdrew: {total_withdrew}")
print(f"Total Discountinued: {total_discontinued}")

In [ ]:
#October 2nd - Code For DF to print Retention Rate for ALL Students and for TRANSFERRED Students

In [ ]:
print("This Cell Calculates the Retention Rates For a Given Start and End Semester. ")
print("It first calculates the rentention rate for all students of the college, ")
print("then the students who transferred into the college.")
print("This method tries to consider discontinued as graduated but fails to be accurate by showing 110% for a case.")

def calculate_retention_rate_per_college(df, term_start, term_end):
    # Get unique colleges from the college_et column
    colleges = df['college_et'].dropna().unique()
    results = {}

    for college in colleges:
        # Filter for students in the specified college at the start term
        students_start = df[(df['term'] == term_start) & (df['college_et'] == college)] #start term and college match
        total_students_start = len(students_start)

        # Find students at the end term
        students_end = df[df['term'] == term_end] #students at end

        # Identify retained students who were in the college at start term
        retained_students = students_end[students_end['encrypt_id'].isin(students_start['encrypt_id'])] #get those who stayed
        total_retained = len(retained_students)

        # Calculate withdrew students for the end term for that college
        withdrew_students = students_end[(students_end['withdrew'] == 1) & (students_end['college_et'] == college)]
        total_withdrew = len(withdrew_students) #get those who withdrew

        # Calculate discontinued students for the end term for that college
        discontinued_students = students_end[(students_end['discontinued'] == 1) & (students_end['college_et'] == college)]
        total_discontinued = len(discontinued_students) #get those who discontinued

        # Calculate retention rate
        effective_start = total_students_start - total_discontinued #(total - disontinued for more accuracy)
        retention_rate = (total_retained / effective_start) * 100 if effective_start > 0 else 0

        # Store results in a dictionary
        results[college] = {
            'Total Students at Start Term': total_students_start,
            'Total Retained': total_retained,
            'Total Withdrew': total_withdrew,
            'Total Discontinued': total_discontinued,
            'Retention Rate': retention_rate
        }

    return results


def calculate_transfers_into_college(df, term_start, term_end):
    print(" ")
    print("Method: using only students who transferred into the college.")
    
    # Get unique colleges from the college_et column
    colleges = df['college_et'].dropna().unique()
    results = {}

    for college in colleges:
        # Find students who were not in the college at the start term but are at the end term
        students_start = df[(df['term'] == term_start)]
        students_end = df[(df['term'] == term_end) & (df['college_et'] == college)]

        # Identify students who transferred into the college (did not start in college but ended in college)
        # Example: Fall 2021 - SM but Spring 2022 - LA
        transferred_students = students_end[~students_end['encrypt_id'].isin(students_start[students_start['college_et'] == college]['encrypt_id'])]
        total_transferred = len(transferred_students)

        # Calculate retained students from those who transferred into the college
        retained_students = transferred_students[transferred_students['encrypt_id'].isin(students_start['encrypt_id'])]
        total_retained = len(retained_students)

        # Calculate withdrew students for the end term for that college
        withdrew_students = students_end[(students_end['withdrew'] == 1) & (students_end['college_et'] == college)]
        total_withdrew = len(withdrew_students)

        # Calculate discontinued students for the end term for that college
        discontinued_students = students_end[(students_end['discontinued'] == 1) & (students_end['college_et'] == college)]
        total_discontinued = len(discontinued_students)

        # Calculate retention rate
        retention_rate = (total_retained / total_transferred) * 100 if total_transferred > 0 else 0

        # Store results in a dictionary
        results[college] = {
            'Total Transferred into College': total_transferred,
            'Total Retained': total_retained,
            'Total Withdrew': total_withdrew,
            'Total Discontinued': total_discontinued,
            'Retention Rate for Transferred Students': retention_rate
        }

    return results


# Example usage
start_term = 202210  # Start term
end_term = 202220    # End term

# Calculate retention rates for all students
retention_results = calculate_retention_rate_per_college(outcome_df, start_term, end_term)

# Display the results
for college, stats in retention_results.items():
    print(f"\nCollege: {college}")
    print(f"Total Students at Start Term: {stats['Total Students at Start Term']}")
    print(f"Total Retained: {stats['Total Retained']}")
    print(f"Total Withdrew: {stats['Total Withdrew']}")
    print(f"Total Discontinued: {stats['Total Discontinued']}")
    print(f"Retention Rate: {stats['Retention Rate']:.2f}%")

# Calculate retention rates for students who transferred into the college
transfer_results = calculate_transfers_into_college(outcome_df, start_term, end_term)

# Display the results for transferred students
for college, stats in transfer_results.items():
    print(f"\nCollege: {college}")
    print(f"Total Transferred into College: {stats['Total Transferred into College']}")
    print(f"Total Retained: {stats['Total Retained']}")
    print(f"Total Withdrew: {stats['Total Withdrew']}")
    print(f"Total Discontinued: {stats['Total Discontinued']}")
    print(f"Retention Rate for Transferred Students: {stats['Retention Rate for Transferred Students']:.2f}%")

    import pandas as pd

# Assuming the functions from previous code are already defined...

start_term = 202110  # Start term
end_term = 202120    # End term

# Calculate retention rates for all students
retention_results = calculate_retention_rate_per_college(outcome_df, start_term, end_term)

# Calculate retention rates for students who transferred into the college
transfer_results = calculate_transfers_into_college(outcome_df, start_term, end_term)

# Create a summary DataFrame
summary_data = []

# Combine results into a summary list
for college in retention_results.keys():
    college_data = {
        'College': college,
        'Retention Rate for ALL Students in College': retention_results[college]['Retention Rate'],
        'Retention Rate for Students SWITCHED to College': transfer_results[college]['Retention Rate for Transferred Students']
    }
    summary_data.append(college_data)

# Create DataFrame from the summary data
summary_df = pd.DataFrame(summary_data)

# Display the summary DataFrame
summary_df


In [ ]:
print("This Cell Calculates the Retention Rates For a Given Start and End Semester. ")
print("It first calculates the rentention rate for all students of the college, ")
print("then the students who transferred into the college.")
print("This method does not consider discontinued so values may be a little lower than actual retention.")
def calculate_retention_rate_per_college(df, term_start, term_end):
    # Get unique colleges from the college_et column
    colleges = df['college_et'].dropna().unique()
    results = {}

    for college in colleges:
        # Filter for students in the specified college at the start term
        students_start = df[(df['term'] == term_start) & (df['college_et'] == college)] #start term and college match
        print(students_start)
        total_students_start = len(students_start)

        # Find students at the end term
        students_end = df[df['term'] == term_end] #students at end

        # Identify retained students who were in the college at start term
        retained_students = students_end[students_end['encrypt_id'].isin(students_start['encrypt_id'])] #get those who stayed
        total_retained = len(retained_students)

        # Calculate withdrew students for the end term for that college
        withdrew_students = students_end[(students_end['withdrew'] == 1) & (students_end['college_et'] == college)]
        total_withdrew = len(withdrew_students) #get those who withdrew

        # Calculate discontinued students for the end term for that college
        #discontinued_students = students_end[(students_end['discontinued'] == 1) & (students_end['college_et'] == college)]
        #total_discontinued = len(discontinued_students) #get those who discontinued

        # Calculate retention rate
        effective_start = total_students_start #(total - disontinued for more accuracy)
        retention_rate = (total_retained / effective_start) * 100 if effective_start > 0 else 0

        # Store results in a dictionary
        results[college] = {
            'Total Students at Start Term': total_students_start,
            'Total Retained': total_retained,
            'Total Withdrew': total_withdrew,
            #'Total Discontinued': total_discontinued,
            'Retention Rate': retention_rate
        }

    return results


def calculate_transfers_into_college(df, term_start, term_end):
    print(" ")
    print("Method: using only students who transferred into the college.")
    
    # Get unique colleges from the college_et column
    colleges = df['college_et'].dropna().unique()
    results = {}

    for college in colleges:
        # Find students who were not in the college at the start term but are at the end term
        students_start = df[(df['term'] == term_start)]
        students_end = df[(df['term'] == term_end) & (df['college_et'] == college)]

        # Identify students who transferred into the college (did not start in college but ended in college)
        # Example: Fall 2021 - SM but Spring 2022 - LA
        transferred_students = students_end[~students_end['encrypt_id'].isin(students_start[students_start['college_et'] == college]['encrypt_id'])]
        total_transferred = len(transferred_students)

        # Calculate retained students from those who transferred into the college
        retained_students = transferred_students[transferred_students['encrypt_id'].isin(students_start['encrypt_id'])]
        total_retained = len(retained_students)

        # Calculate withdrew students for the end term for that college
        withdrew_students = students_end[(students_end['withdrew'] == 1) & (students_end['college_et'] == college)]
        total_withdrew = len(withdrew_students)

        # Calculate discontinued students for the end term for that college
        #discontinued_students = students_end[(students_end['discontinued'] == 1) & (students_end['college_et'] == college)]
        #total_discontinued = len(discontinued_students)

        # Calculate retention rate
        retention_rate = (total_retained / total_transferred) * 100 if total_transferred > 0 else 0

        # Store results in a dictionary
        results[college] = {
            'Total Transferred into College': total_transferred,
            'Total Retained': total_retained,
            'Total Withdrew': total_withdrew,
            #'Total Discontinued': total_discontinued,
            'Retention Rate for Transferred Students': retention_rate
        }

    return results


# Example usage
start_term = 202210  # Start term
end_term = 202220    # End term

# Calculate retention rates for all students
retention_results = calculate_retention_rate_per_college(outcome_df, start_term, end_term)

# Display the results
for college, stats in retention_results.items():
    print(f"\nCollege: {college}")
    print(f"Total Students at Start Term: {stats['Total Students at Start Term']}")
    print(f"Total Retained: {stats['Total Retained']}")
    print(f"Total Withdrew: {stats['Total Withdrew']}")
   # print(f"Total Discontinued: {stats['Total Discontinued']}")
    print(f"Retention Rate: {stats['Retention Rate']:.2f}%")

# Calculate retention rates for students who transferred into the college
transfer_results = calculate_transfers_into_college(outcome_df, start_term, end_term)

# Display the results for transferred students
for college, stats in transfer_results.items():
    print(f"\nCollege: {college}")
    print(f"Total Transferred into College: {stats['Total Transferred into College']}")
    print(f"Total Retained: {stats['Total Retained']}")
    print(f"Total Withdrew: {stats['Total Withdrew']}")
    #print(f"Total Discontinued: {stats['Total Discontinued']}")
    print(f"Retention Rate for Transferred Students: {stats['Retention Rate for Transferred Students']:.2f}%")
    
import pandas as pd

# Assuming the functions from previous code are already defined...

start_term = 202110  # Start term
end_term = 202120    # End term

# Calculate retention rates for all students
retention_results = calculate_retention_rate_per_college(outcome_df, start_term, end_term)

# Calculate retention rates for students who transferred into the college
transfer_results = calculate_transfers_into_college(outcome_df, start_term, end_term)

# Create a summary DataFrame
summary_data = []

# Combine results into a summary list
for college in retention_results.keys():
    college_data = {
        'College': college,
        'Retention Rate for ALL Students in College': retention_results[college]['Retention Rate'],
        'Retention Rate for Students SWITCHED to College': transfer_results[college]['Retention Rate for Transferred Students']
    }
    summary_data.append(college_data)

# Create DataFrame from the summary data
summary_df = pd.DataFrame(summary_data)

# Display the summary DataFrame
summary_df



In [ ]:
# 2. Look at discountinued rates per term to see if any improvement is occuring (rates are lessening). 
# 3. test any correlation between number of major changes for students who have discountinued from the university

In [ ]:
# now that we have a function to find the term, major switched, and gpa for the semester
# 1. label which term in students SFA career did they switch (1st, 2nd, 5th, etc?)
    # everyone starts/ends with different terms so consider that
# 2. random sample of students (need to find a way to measure gpa change for students for each semester)

In [ ]:
outcome_df.corr()